# 指数轮动策略new

In [ ]:
from tick_trade_api import TradeAPI
# 输入指定账户
# trade_api=TradeAPI('376000093011',order_policy=MarketPolicy)
trade_api=TradeAPI('79644908',order_policy=MarketPolicy)


In [ ]:
# 指数轮动策略
import pandas as pd
#初始化函数
def init(context):   
    # 按月交易第一个交易日
    run_monthly(trade,date_rule=1)
    # 输入需要交易的ETF
    context.security = ['159901.OF', # 深证100ETF
                        '510050.OF', # 上证50ETF
                        '159915.OF', # 创业板ETF
                        '510300.OF', # 沪深300ETF
                        '510500.OF', # 中证500ETF
                        '510180.OF', # 上证180ETF
                        '159902.OF', # 中小板100ETF
                        '159905.OF', # 深红利ETF
                        ]
    # 指数强度系数（天）
    context.N = 20
#交易函数
def trade(context,bar_dict):
    # 获取指数行情数据
    price = history(context.security,['close'],context.N,'1d',True,'pre',is_panel=1)
    # 获取收盘价
    df = price['close']
    # 计算强弱
    indexreturn = df.iloc[-1]/df.iloc[0]-1
    # 排序
    indexreturn = indexreturn.sort_values()
    # 选取最强指数及强弱值
    index_T = indexreturn.iloc[-1]
    index = list(indexreturn.index)[-1]
    
    if index_T>0:
        # 获取当前持仓
        holdindex = list(context.portfolio.stock_account.positions.keys())
        if len(holdindex)<1:
            # 买入
            order_target_percent(index,1)
            print('最强指数{},强弱度{},买入'.format(index,round(index_T,4)))
        else:
            for i in holdindex:
                if i == index:
                    print('==> i == index')
                else:
                    # 清仓
                    order_target(i, 0)
                    # 买入
                    order_target_percent(index,1)
                    print('最强指数{},强弱度{},买入'.format(index,round(index_T,4)))
            
#设置风控
def handle_bar(context,bar_dict):
    # 获取指数行情数据
    price = history(context.security,['close'],context.N,'1d',True,'pre',is_panel=1)
    # 获取收盘价
    df = price['close']
    # 计算强弱
    indexreturn = df.iloc[-1]/df.iloc[0]-1
    # 排序
    indexreturn = indexreturn.sort_values()
    # 选取最强指数及强弱值
    index_T = indexreturn.iloc[-1]
    index = list(indexreturn.index)[-1]
    # 获取当前持仓
    holdindex = list(context.portfolio.stock_account.positions.keys())
    # 择时判定
    if index_T<=0:
        for i in holdindex:
            # 清仓
            order_target(i, 0)
            # 打印
            print('最强指数强弱度{},清仓'.format(index_T))
            
            

In [ ]:
# 系统重置
try:
    import shutil
    shutil.rmtree('persist/研究环境策略')
except:
    pass

In [ ]:
# 模拟运行
#rtrade = research_trade('研究环境策略', source_code, capital_base=float(10000000), frequency='MINUTE', trade_api=trade_api)#trade_api可传入List[TradeAPI]，实现多账户交易


In [ ]:
# 回测运行
# research_strategy(source_code, start_date=None, end_date=None, capital_base=100000, frequency='DAILY', stock_market='STOCK', benchmark=None)
#research_strategy(source_code, start_date='20191230', end_date='20220527', capital_base=float(1000000), frequency='MINUTE', stock_market='STOCK', benchmark=None)
